In [ ]:
!python -m spacy download en
!python -m spacy download de
!python -m spacy download es
!python -m spacy download fr

In [ ]:
import re
import nltk
import spacy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
from nltk.corpus import stopwords

from os import path
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
from glob import glob

stopwords = set(STOPWORDS)

In [ ]:
def langcheck(lang):
    if lang == 'en':
        return 'english'
    elif lang == 'es':
        return 'spanish'
    elif lang == 'de':
        return 'german'
    elif lang == 'fr':
        return 'french'
    else:
        raise Exception("Sorry, we don't support this language")

def create_cloud(file, lang):
    file = pd.read_csv(file, 'rb', delimiter = '\t')
    
    nlp = spacy.load(lang)

    stopwordsList = set(nltk.corpus.stopwords.words(langcheck(lang))) # load list of stopwords of target language
    stopwordsList.add('covid') # here we add some covid-related words
    stopwordsList.add('covid19')
    stopwordsList.add('corona')
    stopwordsList.add('coronavirus')
    stopwordsList.add('covid-19')
    stopwordsList.add('-PRON-')

    text = '' # create an empty string
    for row in file['text']:
        row = re.sub('((www\.[\s]+)|(https?://[^\s]+))', ' ', row)
        row = re.sub('@[A-Za-z0-9_-]+', ' ', row)
        row = re.sub('RT', ' ', row)
        row = re.sub('#', ' ', row)
        row = re.sub('[^\w\s]', ' ', row)
        row = re.sub('[0-9]', ' ', row)
        text = text + row # collecting preprocessed texts in one string
        
    text = text.lower() # prepare for tokenization
    text_content = nlp(text)
    text_content = [token.lemma_ for token in text_content]
    

    text_content = [word for word in text_content if word not in stopwordsList] # deleting stopwords in text
    text_content = [s for s in text_content if len(s) >= 4] # filter words with lenght smaller than ...

    fdist = nltk.FreqDist(text_content)   
        
    word_dict = {}
    for key in fdist:
        word_dict[key] = fdist[key]  

    WC_height = 500
    WC_width = 1000
    WC_max_words = 200
    wordCloud = WordCloud(stopwords=stopwords, width=1000, height=500, max_font_size=100, 
                            max_words=80, background_color="white")
    wordCloud.generate_from_frequencies(word_dict)
        
    plt.figure()
    plt.imshow(wordCloud)
    plt.axis("off")
    plt.show()

In [ ]:
create_cloud('data/out/gobmx.csv', 'es') # support of 'en', 'es', 'de', 'fr' languages
create_cloud('data/out/govuk.csv', 'en')